In [2]:
import urllib
import urllib.request
import urllib.parse
from urllib.request import urlopen 
from urllib.request import urlretrieve 

from datetime import datetime
import time,locale

from bs4 import BeautifulSoup
import re
import unicodedata

import pandas as pd
import numpy as np

In [2]:
player_fields = ['license','name','surname','gender']
dfPlayer = pd.DataFrame(columns = player_fields)


In [4]:
df = pd.read_csv('badiste_Club.csv', encoding = 'iso-8859-15', sep=';')
df.drop(columns = df.columns[range(4,17)], inplace=True)
df.columns = ['name','short_name','dep','city_name']
df.drop(columns='dep', inplace=True)
df.index.names = ['id']

df["name_search"] = df["name"] + " (" + df["short_name"] + ")"
df.to_csv('Club.csv', columns = ['name','short_name','city_name','lat','long','url'])
def getClubId(search_str, df):
    return df.index[df['name_search'] == search_str].tolist()

/Users/Karine/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


## This cell takes a very long time to run if the players are all fetched at once

In [4]:
player_fields = ['license','name','surname','gender', 'S', 'D', 'M', 'Moy', 'club_id']
dfPlayer = pd.DataFrame(columns = player_fields)

c = 0
wrongClubs = []
pl = 0
for c in range(len(df) - 1): #to remove the fédération
    if (c %10 == 0):
            print(c)
    clubFullName = df.at[c, 'name']
    sigle = df.at[c, 'short_name']
    lowName = unicodedata.normalize('NFD', sigle).encode('ascii', 'ignore').decode('utf-8').lower()
    if(not ('comite' in lowName or 'ligue' in lowName)):
        
        try: 
            playersLink = 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=&sigle='+ lowName +'&lim=1000&Submit=Rechercher'
            dataClub = urllib.parse.urlencode({'date' : '100'}).encode("utf-8")

            reqClub = urllib.request.Request(playersLink)
            with urllib.request.urlopen(reqClub,data=dataClub) as f:
                respClub = f.read()

            #codeSource = html.read().decode('utf-8')
            soupClub = BeautifulSoup(respClub, 'html.parser')
            genders = {'H': 0, 'F': 1}
            trsClub = soupClub.find_all("tr", {"class": "tux"})
            
            player_count = 0
            for player in trsClub:
                tdsClub = player.find_all('td')
                nameClub = re.search('(.[^A-Z]+)+([A-Z ]+)', tdsClub[0].text)
                playerName = nameClub.group(1).strip()
                playerSurname = nameClub.group(2).strip()
                if(not 'anonyme' in playerName.lower()):
                    S = tdsClub[3].text
                    D = tdsClub[4].text
                    M = tdsClub[5].text
                    Moy = tdsClub[9].text
                    playerLicense = tdsClub[10].text

                    playerGender = genders[tdsClub[1].text]

                    dfPlayer.loc[pl] = [playerLicense, playerName, playerSurname, playerGender, S, D, M, Moy, c]
                    pl = pl + 1
        except:
            wrongClubs.append(sigle)
        time.sleep(0.2)
        c = c + 1
dfPlayer.to_csv('player.csv', index=False)
wrongDf = pd.DataFrame({'col':wrongClubs})
wrongDf.to_csv('wrong_club_names.csv', index=False)

2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200


## There are some players that are duplicated because some clubs had the same short names, so we need to clean that 

In [6]:



df = pd.read_csv('badiste_Club.csv', encoding = 'iso-8859-15', sep=';')
df.drop(columns = df.columns[range(4,17)], inplace=True)
df.columns = ['name','short_name','dep','city_name']
df.drop(columns='dep', inplace=True)
#df.index.names = ['id']
df['id'] = range(0, len(df))

df["name_search"] = df["name"] + " (" + df["short_name"] + ")"
df.to_csv('Club.csv', columns = ['name','short_name','city_name','lat','long','url'])
def getClubId(search_str, df):
    return df.index[df['name_search'] == search_str].tolist()


ligues = [234, 375, 418, 437, 452, 468, 473, 480, 
          541, 636, 718, 729, 731, 738, 881, 1076, 
          1232, 1252, 1840]

newDf = df[['id','short_name', 'city_name']]
newDf['newCol'] = list(zip(newDf['id'], newDf['city_name']))
newDf = newDf[['short_name', 'newCol']]

a = dict(newDf.set_index('short_name').groupby(level = 0).apply(lambda x : x.to_dict(orient= 'records')))
a = {k: [v2.get('newCol') for v2 in v] for k, v in a.items()}

dfPlayer = pd.read_csv('player.csv', dtype={'club_id': str})

duplicate_keys = [k for k, v in a.items() if len(v) > 1]

print(len(duplicate_keys))
#len = 206
print(duplicate_keys)
#print(a.get('AB13'))
for k in duplicate_keys:
    cities = {}
    for (id, c) in a.get(k):
        if(c in cities):
            print(c)
            print(k)
        else:
            cities[c] = 1





wrongPlayers = pd.DataFrame()
for nb in range(len(duplicate_keys)):
    wrongs = a.get(duplicate_keys[nb])
    for id, city in wrongs:
        
        wrongPlayers = wrongPlayers.append(dfPlayer.loc[dfPlayer['club_id'] == str(id)])
        dfPlayer = dfPlayer.loc[dfPlayer['club_id'] != str(id)]
        
wrongPlayers.to_csv('wrongPlayers.csv', index=False)
dfPlayer.to_csv('correctPlayer.csv', index=False)

player_fields = ['license','name','surname','gender', 'S', 'D', 'M', 'Moy', 'club_id']
dfPlayer = pd.DataFrame(columns = player_fields)

fetchNumber = 0
wrongClubs = []
pl = 0
for c in range(len(duplicate_keys)):
    k = duplicate_keys[c]
    
    if(k != 'CD04'):
        
        for (id, city) in a.get(k):
            

            lowName = unicodedata.normalize('NFD', k).encode('ascii', 'ignore').decode('utf-8').lower()
            town = unicodedata.normalize('NFD', city).encode('ascii', 'ignore').decode('utf-8').lower()

            try: 
                
                playersLink = 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville='+town+'&sigle='+ lowName +'&lim=1000&Submit=Rechercher'
                dataClub = urllib.parse.urlencode({'date' : '100'}).encode("utf-8")

                reqClub = urllib.request.Request(playersLink)
                with urllib.request.urlopen(reqClub,data=dataClub) as f:
                    respClub = f.read()

                #codeSource = html.read().decode('utf-8')
                soupClub = BeautifulSoup(respClub, 'html.parser')
                genders = {'H': 0, 'F': 1}
                trsClub = soupClub.find_all("tr", {"class": "tux"})
                
                player_count = 0
                for player in trsClub:
                    tdsClub = player.find_all('td')
                    nameClub = re.search('(.[^A-Z]+)+([A-Z ]+)', tdsClub[0].text)
                    playerName = nameClub.group(1).strip()
                    playerSurname = nameClub.group(2).strip()
                    if(not 'anonyme' in playerName.lower()):
                        S = tdsClub[3].text
                        D = tdsClub[4].text
                        M = tdsClub[5].text
                        Moy = tdsClub[9].text
                        playerLicense = tdsClub[10].text

                        playerGender = genders[tdsClub[1].text]

                        dfPlayer.loc[pl] = [playerLicense, playerName, playerSurname, playerGender, S, D, M, Moy, id]
                        pl = pl + 1
                
                print(c)
            except:
                wrongClubs.append(lowName)
            time.sleep(0.2)
    
dfPlayer.to_csv('duplicatePlayers.csv', index=False)
wrongDf = pd.DataFrame({'col':wrongClubs})
wrongDf.to_csv('duplicateWrong_club.csv', index=False)


## There were two clubs with the same name in the same city, so we cleaned them by hand

In [1]:


df = pd.read_csv('badiste_Club.csv', encoding = 'iso-8859-15', sep=';')
df.drop(columns = df.columns[range(4,17)], inplace=True)
df.columns = ['name','short_name','dep','city_name']
df.drop(columns='dep', inplace=True)
#df.index.names = ['id']
df['id'] = range(0, len(df))

df["name_search"] = df["name"] + " (" + df["short_name"] + ")"
df.to_csv('Club.csv', columns = ['name','short_name','city_name','lat','long','url'])
def getClubId(search_str, df):
    return df.index[df['name_search'] == search_str].tolist()

newDf = df[['id','short_name', 'city_name']]
newDf['newCol'] = list(zip(newDf['id'], newDf['city_name']))
newDf = newDf[['short_name', 'newCol']]

a = dict(newDf.set_index('short_name').groupby(level = 0).apply(lambda x : x.to_dict(orient= 'records')))
a = {k: [v2.get('newCol') for v2 in v] for k, v in a.items()}

names = [('abc', 'Les Aix-d\'Angillon'),
('abc', 'La Chapelle-d\'Armentières'),
('abm', 'Mons en Baroeul'),
('alpb', 'Le Palais-sur-Vienne'),
('altb', 'La Teste-de-Buch'),
('arb', 'Auriol / Roquevaire'),
('asbl', 'Le Loroux-Bottereau'),
('asc', 'LES EGLISOTTES'),
('asmp', 'Le Perreux-sur-Marne'),
('asmr', 'Le Mesnil-le-Roi'),
('asrb', 'La Rochette'),
('avt', 'Taverny/Saint Leu la Forêt'),
('avt', 'Le Temple-de-Bretagne'),
('bam', 'Le Monastère'),
('bam', 'Tarascon et St Rémy de Provence'),
('bb', 'Pays de Belvès'),
('bb', 'Bourg Sur Gironde'),
('bbc', 'Bagnoles De L Orne Normandie'),
('bce', 'La Chapelle-sur-Erdre'),
('bcg', 'La Garde'),
('bch', 'La Haye-du-Puits'),
('bcl', 'Le Luc En Provence'),
('bclc', 'Le Chesnay'),
('bclc', 'Le Châtelet'),
('bcm', 'Ste Clotilde'),
('bcn', 'NOUVION EN PONTHIEU'),
('bcp', 'La Possession'),
('bcp', 'La Plaine-des-Palmistes'),
('blc', 'La Châtre'),
('bp', 'Plerin Sur Mer'),
('bs', 'La Garenne-Colombes'),
('bt', 'Le Theil'),
('cbe', 'Esvre Sur Indre'),
('cbm', 'LE MALESHERBOIS'),
('cbs', 'Scey sur Saone'),
('clbf', 'La Frénaye'),
('essltb', 'St Lambert Des Levees'),
('fdv', 'La Meilleraie-Tillay'),
('hbc', 'Le Havre'),
('lbb', 'La Bresse'),
('lpt', 'Le Tampon'),
('lvb', 'Le Barp'),
('lvb', 'Le Buisson-de-Cadouin'),
('lvc', 'La Chapelle-des-Fougeretz'),
('lvs', 'Le Sap'),
('mbc', 'Maure de Bretagne - Val d\'Anast'),
('mbc', 'Les Martres-de-Veyre'),
('pb', 'Le Plessis-Pâté'),
('pb', 'Ploeuc Sur Lie'),
('sab', 'La Tour-d\'Auvergne'),
('sobad', 'L\'Île-d\'Olonne,Vairé,Les Sables'),
('usc', 'La Crau'),
('uscb', 'Le Châtelet-en-Brie'),
('uscm', 'Mézidon Vallée d\'Auge'),
('usfb', 'La Ferrière'),
('usp', 'Le Pecq'),
('vb', 'La Bassée')]



namesAndIds = []
for s, v in names:
    idsVilles = a.get(s.upper())
    id = [i for i, vi in idsVilles if( vi.lower() == v.lower())]
    namesAndIds.append((s, id, v))

urls = ["http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=Les+Aix-d%27Angillon&sigle=abc&lim=1000&Submit=Rechercher",
 "http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+chapelle-d'armentieres&sigle=abc&lim=1000&Submit=Rechercher",
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=mons+en+baroeul&sigle=abm&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+palais-sur-vienne&sigle=alpb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+teste-de-buch&sigle=altb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=auriol+/+roquevaire&sigle=arb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+loroux-bottereau&sigle=asbl&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=les+eglisottes&sigle=asc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+perreux-sur-marne&sigle=asmp&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+mesnil-le-roi&sigle=asmr&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+rochette&sigle=asrb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=taverny/saint+leu+la+foret&sigle=avt&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+temple-de-bretagne&sigle=avt&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+monastere&sigle=bam&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=tarascon+et+st+remy+de+provence&sigle=bam&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=pays+de+belves&sigle=bb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=bourg+sur+gironde&sigle=bb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=bagnoles+de+l+orne+normandie&sigle=bbc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+chapelle-sur-erdre&sigle=bce&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+garde&sigle=bcg&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+haye-du-puits&sigle=bch&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+luc+en+provence&sigle=bcl&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+chesnay&sigle=bclc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+chatelet&sigle=bclc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=ste+clotilde&sigle=bcm&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=nouvion+en+ponthieu&sigle=bcn&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+possession&sigle=bcp&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+plaine-des-palmistes&sigle=bcp&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+chatre&sigle=blc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=plerin+sur+mer&sigle=bp&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+garenne-colombes&sigle=bs&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+theil&sigle=bt&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=esvre+sur+indre&sigle=cbe&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+malesherbois&sigle=cbm&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=scey+sur+saone&sigle=cbs&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+frenaye&sigle=clbf&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=st+lambert+des+levees&sigle=essltb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+meilleraie-tillay&sigle=fdv&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+havre&sigle=hbc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+bresse&sigle=lbb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+tampon&sigle=lpt&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+barp&sigle=lvb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+buisson-de-cadouin&sigle=lvb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+chapelle-des-fougeretz&sigle=lvc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+sap&sigle=lvs&lim=1000&Submit=Rechercher',
 "http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=maure+de+bretagne+-+val+d'anast&sigle=mbc&lim=1000&Submit=Rechercher",
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=les+martres-de-veyre&sigle=mbc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+plessis-pate&sigle=pb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=ploeuc+sur+lie&sigle=pb&lim=1000&Submit=Rechercher',
 "http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+tour-d'auvergne&sigle=sab&lim=1000&Submit=Rechercher",
 "http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=l'ile-d'olonne,vaire,les+sables&sigle=sobad&lim=1000&Submit=Rechercher",
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+crau&sigle=usc&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+chatelet-en-brie&sigle=uscb&lim=1000&Submit=Rechercher',
 "http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=mezidon+vallee+d'auge&sigle=uscm&lim=1000&Submit=Rechercher",
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+ferriere&sigle=usfb&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=le+pecq&sigle=usp&lim=1000&Submit=Rechercher',
 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=la+bassee&sigle=vb&lim=1000&Submit=Rechercher']

player_fields = ['license','name','surname','gender', 'S', 'D', 'M', 'Moy', 'club_id']
dfPlayer = pd.DataFrame(columns = player_fields)

fetchNumber = 0
wrongClubs = []
pl = 0
for i in range(len(urls)):

    try: 

        playersLink = urls[i]
        s, id, v = namesAndIds[i]
        dataClub = urllib.parse.urlencode({'date' : '100'}).encode("utf-8")

        reqClub = urllib.request.Request(playersLink)
        with urllib.request.urlopen(reqClub,data=dataClub) as f:
            respClub = f.read()

        #codeSource = html.read().decode('utf-8')
        soupClub = BeautifulSoup(respClub, 'html.parser')
        genders = {'H': 0, 'F': 1}
        trsClub = soupClub.find_all("tr", {"class": "tux"})

        player_count = 0
        for player in trsClub:
            tdsClub = player.find_all('td')
            nameClub = re.search('(.[^A-Z]+)+([A-Z ]+)', tdsClub[0].text)
            playerName = nameClub.group(1).strip()
            playerSurname = nameClub.group(2).strip()
            if(not 'anonyme' in playerName.lower()):
                S = tdsClub[3].text
                D = tdsClub[4].text
                M = tdsClub[5].text
                Moy = tdsClub[9].text
                playerLicense = tdsClub[10].text

                playerGender = genders[tdsClub[1].text]

                dfPlayer.loc[pl] = [playerLicense, playerName, playerSurname, playerGender, S, D, M, Moy, id[0]]
                pl = pl + 1

    except:
        wrongClubs.append(lowName)
    time.sleep(0.2)
    
dfPlayer.to_csv('lastPlayers.csv', index=False)
wrongDf = pd.DataFrame({'col':wrongClubs})
wrongDf.to_csv('lastWrongs.csv', index=False)

fout=open("correctPlayer.csv","a")
# first file:
for line in open("lastPlayers.csv"):
    fout.write(line)
    
fout.close()


dfPlayerTest = pd.read_csv('correctPlayer.csv', dtype={'license': str, 'gender': str, 'Moy': str, 'club_id': str})

club2191 = ['00134747', '07010819' , '00504031' , '00269396' , '06568916' ,'06467595','06633582' ,'06786629' ,
'06558583','00551676','06929112','06899127','06886583','06820016','00058247','06682342','07215733',
'06870999','06899144','06779566','06733592','06460249','06823833','00395770','06941541','06820052',
'06564016','06705330','00551677','00484053','06786636','00330345','00343951','00384393','06825438',
'06467619','00320592','06820004','06743312','00325035','06642903','00453774','00277990','00565316',
'06752226','00505448','00542997','06573878','06880639','06684214','00058359','06821794','07030764',
'06683933','06529344','06899139','06743772','00454696','06899096','00527017','06894986','00054530',
'06947829','06872297','06480885','00399545','06841062','06692311','06827063','06752222','07143641',
'06588836','06681788','06899146','06642891','06820049','07015559','06980797','06584044','07214972',
'06911596','00424749','06683103','00567421','06899094','00341817','06775265','06840834','06776868',
'07027801','06776872','07215734','07123760','07209086','07214976','07253974','07214982','07214988',
'07123747','07185186','07253973','07209090','07214985','07214984','07185182','07107529','07214980',
'07214978','07185187','07133593','07214986','07209102','07228506','07107735','07214977','06931566',
'07214987','07027809','07214979','07221400','06859345','07030439','06931586','07214973','06976407',
'07185201','07214981','07209099','07209094','07144026','07209092','07185168','06931575','07253975',
'06931584','07107767','07253976','07027802','07214974','07209108','06931568','07214983','07073202',
'07214975','06776881','07107518','06974822','07043325','06820017','06931569','07228511','07215737',
'06931567','06974599','06974573','07107629','06931581','07107834','07185189','06931560','07214971',
'07221404','06853849','06776870','07253969','07247090','07253972','07214970','07221403','07157927',
'07214961','06686260','07214965','07215736','07214962','07253970','00534758','07214956','07157857',
'07214952','06730822','07221405','07228507','07214953','07228510','06495062','07214966','00555841',
'07214967','06878469','07157887','07214951','07228508','07215735','07214954','07247092','07157919',
'07247091','07247089','07067502','07214957','06571463','07214950','00399377','07228509','07157825',
'07214963','07096199','07096122','07128446','05900467','07253971','00370914','07247093','00321463',
'07157849','07214969','07067493','07214964','06662950','07214958','07214959','07214960','06980853',
'06907009','07164531','06620667','07221402','06899100','06899113','07110809','07228512','06940003',
'07221401','06931607','06796752','00318097','06899110','06796718','07164501','07214968','06997856',
'06931592']

dfLast = dfPlayerTest.loc[dfPlayerTest['club_id'] != str(2191)]
dfLast = dfLast.loc[dfLast['club_id'] != str(2192)]

divide = dfPlayerTest.loc[dfPlayerTest['club_id'] == str(2191)]

def newVal(x):
    if(x in club2191):
        return 2191
    else:
        return 2192

divide['club_id'] = divide['license'].map(newVal)
dfLast = dfLast.append(divide)

dfLast.to_csv('correctPlayer.csv')


NameError: name 'pd' is not defined